# File to benchmark the algorithms

## Import dependencies

In [ ]:
from sorters.insertion_sort import (insertion_sort_with_linear_search, insertion_sort_with_binary_searh)
from sorters.binary_tree import BinaryTree
from sorters.avl import AVLTree
from sorters.heap import (heap_sort_iterative, heap_sort_recursive)

from util.data import Data, Order, DATA_PATH

## Constants

In [ ]:
INSERTION_LINEAR: str = "Insertion Sort (linear search)"
INSERTION_BINARY: str = "Insertion Sort (binary search)"
BINARY_TREE: str = "Binary Tree"
AVL_TREE: str = "AVL Tree"
HEAP_ITERATIVE: str = "Heap-Sort (iterative)"
HEAP_RECURSIVE: str = "Heap-Sort (recursive)"

ALGORITHMS: dict = {
    INSERTION_LINEAR: insertion_sort_with_linear_search, 
    INSERTION_BINARY: insertion_sort_with_binary_searh,
    BINARY_TREE: BinaryTree,
    AVL_TREE: AVLTree,
    HEAP_ITERATIVE: heap_sort_iterative,
    HEAP_RECURSIVE: heap_sort_recursive
}

ORDERS: list[Order] = [Order.SORTED, Order.REVERSED, Order.MANY_REPETITIONS, Order.RANDOM]

DATA_SIZES: list[int] = [i for i in range(1_000, 10_500, 500)]
REPETITIONS_PER_ALGORITHM: int = 5

## Creating the simulations

In [ ]:
def main() -> None:
    for algorithm in ALGORITHMS:
        for order in ORDERS:
            for size in DATA_SIZES:
                for _ in range(REPETITIONS_PER_ALGORITHM):
                    data: Data = Data()
                    data.create_data(size, order)

                    if algorithm in [BINARY_TREE, AVL_TREE]:
                        sorter = ALGORITHMS[algorithm]()
                        sorter.insertData(data)
                    else:
                        ALGORITHMS[algorithm](data)

## Run the algorithms

In [ ]:
main()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# ---------------------------------------------------------
# 1. LOAD DATA
# ---------------------------------------------------------
csv_files: dict[str, str] = {
    "Insertion Sort (Linear)": f"{DATA_PATH}/insertion_sort(linear_search).csv",
    "Insertion Sort (Binary)": f"{DATA_PATH}/insertion_sort(binary_search).csv",
    "Binary Tree": f"{DATA_PATH}/binary_tree.csv",
    "AVL Tree": f"{DATA_PATH}/avl_tree.csv",
    "Heap-Sort (Iterative)": f"{DATA_PATH}/heapsort_iterative.csv",
    "Heap-Sort (Recursive)": f"{DATA_PATH}/heapsort_recursive.csv"
}

dfs: list[pd.DataFrame] = []

for algo_name, file_path in csv_files.items():
    try:
        # Load csv
        df: pd.DataFrame = pd.read_csv(file_path)
        
        # Add a algorithm name column
        df['Algorithm'] = algo_name
        
        dfs.append(df)
    except FileNotFoundError:
        print(f"Aviso: Arquivo não encontrado para {algo_name}: {file_path}")

if not dfs:
    raise ValueError("Nenhum dado foi carregado. Verifique os caminhos dos arquivos CSV.")

# Concat all DataFrames
full_df: pd.DataFrame = pd.concat(dfs, ignore_index=True)

# ---------------------------------------------------------
# 2. MEAN OF THE 5 RUNS
# ---------------------------------------------------------
df_mean: pd.DataFrame = full_df.groupby(['Algorithm', 'Order', 'Data Size'], as_index=False)[['Swaps', 'Comparisons', 'Time']].mean()

print("Amostra dos dados processados (Média):")
display(df_mean.head())

# ---------------------------------------------------------
# 3. VISUAL CONFIGURATIONS
# ---------------------------------------------------------
sns.set_theme(style="whitegrid")
metrics: dict[str, str] = {
    "Time": "Tempo de Execução (s)",
    "Swaps": "Número de Trocas",
    "Comparisons": "Número de Comparações"
}

# ---------------------------------------------------------
# 4. MAKE GRAPHICS
# ---------------------------------------------------------
for metric_col, metric_label in metrics.items():
    g = sns.relplot(
        data=df_mean,
        x="Data Size", 
        y=metric_col,
        hue="Algorithm", 
        col="Order",
        kind="line",
        height=4, 
        aspect=1,
        col_wrap=2,
        marker="o"
    )
    
    g.figure.suptitle(f'Comparação de {metric_label} por Tamanho e Ordenação', y=1.02, fontsize=16)
    g.set_axis_labels("Tamanho dos Dados (N)", metric_label)
    g.set_titles("Ordem: {col_name}")
    
    plt.show()

# ---------------------------------------------------------
# 5. SUMMARY TABLE
# ---------------------------------------------------------
max_size = df_mean['Data Size'].max()
print(f"\n--- Tabela Resumo para Tamanho = {max_size} (Média de Tempo) ---")

summary_table = df_mean[df_mean['Data Size'] == max_size].pivot_table(
    index="Algorithm", 
    columns="Order", 
    values="Time"
)

display(summary_table.style.background_gradient(cmap='RdYlGn_r', axis=0).format("{:.6f} s"))

In [ ]:
# ---------------------------------------------------------
# 1. DEFINING GROUPS
# ---------------------------------------------------------
group_slow = [
    "Insertion Sort (Linear)", 
    "Insertion Sort (Binary)"
]

group_fast = [
    "AVL Tree", 
    "Heap-Sort (Iterative)", 
    "Heap-Sort (Recursive)",
    "Binary Tree"
]

# ---------------------------------------------------------
# 2. DATA SEGMENTATION
# ---------------------------------------------------------
df_slow = df_mean[df_mean['Algorithm'].isin(group_slow)]
df_fast = df_mean[df_mean['Algorithm'].isin(group_fast)]

datasets = {
    "Algoritmos Quadráticos - O(n^2)": df_slow,
    "Algoritmos Log-Lineares - O(n log n)": df_fast
}

# ---------------------------------------------------------
# 3. MAKE SPLITED GRAPHICS
# ---------------------------------------------------------
metrics = {
    "Time": "Tempo de Execução (s)",
    "Swaps": "Número de Trocas",
    "Comparisons": "Número de Comparações"
}

for group_name, df_grupo in datasets.items():
    if df_grupo.empty:
        print(f"Aviso: O grupo '{group_name}' está vazio. Verifique os nomes dos algoritmos.")
        continue

    print(f"\n{'='*20} {group_name} {'='*20}")
    
    for metric_col, metric_label in metrics.items():
        # skip empty graphics
        if df_grupo[metric_col].sum() == 0:
            continue
            
        g = sns.relplot(
            data=df_grupo,
            x="Data Size", 
            y=metric_col,
            hue="Algorithm", 
            col="Order",
            kind="line",
            height=3.5, 
            aspect=1.2,
            col_wrap=2,
            marker="o"
        )
        
        # Ajuste de Títulos e Labels
        g.fig.suptitle(f'{metric_label} - {group_name}', y=1.05, fontsize=14, weight='bold')
        g.set_axis_labels("Tamanho (N)", metric_label)
        g.set_titles("{col_name}")
        
        plt.show()

In [ ]:
df_slow = df_mean[df_mean['Algorithm'].isin(group_slow)].copy()
df_fast = df_mean[df_mean['Algorithm'].isin(group_fast)].copy()

bad_condition = (df_mean['Algorithm'] == 'Binary Tree') & \
                (df_mean['Order'].isin(['SORTED', 'REVERSED']))

bt_bad_data = df_mean[bad_condition]


if not bt_bad_data.empty:
    print("Movendo Binary Tree (Sorted/Reversed) para o grupo de Algoritmos Lentos...")
    
    # Adiciona ao grupo lento
    df_slow = pd.concat([df_slow, bt_bad_data])
    
    # Remove do grupo rápido (filtra tudo que NÃO for a condição ruim)
    # Nota: Precisamos filtrar df_fast baseando-se nas colunas, não apenas no índice
    bad_condition_fast = (df_fast['Algorithm'] == 'Binary Tree') & \
                         (df_fast['Order'].isin(['SORTED', 'REVERSED']))
    df_fast = df_fast[~bad_condition_fast]

datasets = {
    "Algoritmos Quadráticos - O(n^2)": df_slow,
    "Algoritmos Log-Lineares - O(n log n)": df_fast
}

metrics = {
    "Time": "Tempo de Execução (s)",
    "Swaps": "Número de Trocas",
    "Comparisons": "Número de Comparações"
}

for grupo_nome, df_grupo in datasets.items():
    if df_grupo.empty:
        print(f"Aviso: O grupo '{grupo_nome}' está vazio. Verifique os nomes dos algoritmos.")
        continue

    print(f"\n{'='*20} {grupo_nome} {'='*20}")
    
    for metric_col, metric_label in metrics.items():
        if df_grupo[metric_col].sum() == 0:
            continue
            
        g = sns.relplot(
            data=df_grupo,
            x="Data Size", 
            y=metric_col,
            hue="Algorithm", 
            col="Order",
            kind="line",
            height=3.5, 
            aspect=1.2,
            col_wrap=2,
            marker="o"
        )
        
        g.figure.suptitle(f'{metric_label} - {grupo_nome}', y=1.05, fontsize=14, weight='bold')
        g.set_axis_labels("Tamanho (N)", metric_label)
        g.set_titles("{col_name}")
        
        plt.show()